<a href="https://colab.research.google.com/github/tlchampion/simpsons_character_recognition/blob/main/notebooks/model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Purpose

Model 1 will serve as a baseline for model comparisons and will be built without any hyperparameter tuning, using default values for all options. 

<br>
The model will incorporate checkpoints and early stopping callbacks.

# Setup Environment

## Mount google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install and import modules

In [2]:
! pip install python-dotenv

In [3]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import PIL 
import PIL.Image
import pathlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sys
sys.path.insert(0,"/content/drive/My Drive/Projects/simpsons_character_recognition/simpsons_images")
from dotenv import find_dotenv, load_dotenv

In [4]:
from d01_data import make_dataset

In [5]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Activation
from tensorflow.keras import Sequential

# Copy image files to drive local to colab instance





In [6]:
%cp -r /content/drive/MyDrive/Projects/simpsons_character_recognition/data /content/data

# Import Data

In [7]:
train_directory = os.path.join(os.getcwd(),'data/images/simpsons_dataset')
test_directory = os.path.join(os.getcwd(),'data/images/simpsons_testset')
test_labels = os.path.join(os.getcwd(),'data/images/test_labels.csv')


In [8]:
datagen, datagen_test, train, validation, test = make_dataset.create_datasets(train_directory, test_directory, test_labels)

Found 16764 images belonging to 42 classes.
Found 4169 images belonging to 42 classes.
Found 990 validated image filenames.


# Setup Model

In [17]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0.001, patience=5, verbose=0,
    mode='auto', baseline=None, restore_best_weights=True
)

checkpoint_filepath = '/content/drive/MyDrive/Projects/simpsons_character_recognition/callbacks'
checkpoints = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath, monitor='val_loss', verbose=0, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None, initial_value_threshold=None
)

log_dir = '/content/drive/MyDrive/Projects/simpsons_character_recognition/logdir'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [18]:
model_1 = Sequential([
  Conv2D(10, 3, activation='relu', input_shape=(224, 224, 3)),
  MaxPool2D(pool_size=2),
  Conv2D(10, 3, activation='relu'),
  MaxPool2D(),
  Conv2D(10, 3, activation='relu'),
  MaxPool2D(),
  Flatten(),
  Dense(42, activation='softmax')
])

# Compile the model
model_1.compile(loss='categorical_crossentropy',
                optimizer=Adam(),
                metrics=['accuracy'])

# Fit the model
history_1 = model_1.fit(train,
                        epochs=200,
                        steps_per_epoch=len(train),
                        validation_data=validation,
                        validation_steps=len(validation),
                        callbacks=[early_stop,checkpoints, tensorboard_callback])

Epoch 1/5
524/524 [==============================] - 73s 138ms/step - loss: 2.7331 - accuracy: 0.2355 - val_loss: 2.5980 - val_accuracy: 0.3015
Epoch 2/5
524/524 [==============================] - 72s 138ms/step - loss: 2.2092 - accuracy: 0.4000 - val_loss: 2.4358 - val_accuracy: 0.3624
Epoch 3/5
524/524 [==============================] - 71s 136ms/step - loss: 1.9631 - accuracy: 0.4625 - val_loss: 2.3393 - val_accuracy: 0.3924
Epoch 4/5
524/524 [==============================] - 71s 136ms/step - loss: 1.7960 - accuracy: 0.5093 - val_loss: 2.3205 - val_accuracy: 0.3958
Epoch 5/5
524/524 [==============================] - 71s 136ms/step - loss: 1.6612 - accuracy: 0.5503 - val_loss: 2.3045 - val_accuracy: 0.4111


In [19]:
model_1.save('/content/drive/MyDrive/Projects/simpsons_character_recognition/models/model_1')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Projects/simpsons_character_recognition/models/model_1/assets


In [20]:
%tensorboard --logdir logs/fit


UsageError: Line magic function `%tensorboard` not found.


In [ ]:
history_1.history